In [196]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import statistics
#import missingno as msno

In [197]:
path_file = 'U1401.xlsx'
sheets_excel = pd.read_excel('U1401.xlsx',sheet_name=None)

In [198]:
for name in list(sheets_excel.keys()):
    sheets_excel[name].to_csv(name+'.csv', index=False)

### P4_S01

In [199]:
DF_P4_S1 = pd.read_csv('U1401P4S01.csv')

In [200]:
DF_P4_S1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12312 entries, 0 to 12311
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address        12312 non-null  int64  
 1   member         12312 non-null  int64  
 2   employed_w     12312 non-null  object 
 3   ISCO_w         12311 non-null  float64
 4   ISIC_w         12312 non-null  int64  
 5   status_w       12312 non-null  int64  
 6   hours_w        12312 non-null  object 
 7   days_w         12312 non-null  object 
 8   income_w_m     12312 non-null  int64  
 9   income_w_y     12312 non-null  int64  
 10  wage_w_m       12312 non-null  int64  
 11  wage_w_y       12312 non-null  int64  
 12  perk_w_m       12311 non-null  float64
 13  perk_w_y       12311 non-null  float64
 14  netincome_w_m  12312 non-null  int64  
 15  netincome_w_y  12312 non-null  int64  
 16  Fasl           12312 non-null  int64  
 17  year           12312 non-null  int64  
 18  DYCOL0

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون DYcol00 کامل nan است.

In [201]:
DF_P4_S1.drop(['DYCOL00'], axis=1, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون های days_w, hours_w , status_w به int تبدیل می‌کنیم

In [202]:
DF_P4_S1.loc[DF_P4_S1.days_w.isin([' ','  ']),['days_w' ]] = np.nan
DF_P4_S1.loc[DF_P4_S1.hours_w.isin([' ','  ']),['hours_w' ]] = np.nan
DF_P4_S1.loc[DF_P4_S1.status_w.isin([' ','  ']),['status_w' ]] = np.nan

DF_P4_S1.status_w= DF_P4_S1.status_w.astype(pd.Int64Dtype())
DF_P4_S1.days_w= DF_P4_S1.days_w.astype(pd.Int64Dtype())
DF_P4_S1.hours_w= DF_P4_S1.hours_w.astype(pd.Int64Dtype())

In [203]:
DF_P4_S1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12312 entries, 0 to 12311
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address        12312 non-null  int64  
 1   member         12312 non-null  int64  
 2   employed_w     12312 non-null  object 
 3   ISCO_w         12311 non-null  float64
 4   ISIC_w         12312 non-null  int64  
 5   status_w       12312 non-null  Int64  
 6   hours_w        11615 non-null  Int64  
 7   days_w         11615 non-null  Int64  
 8   income_w_m     12312 non-null  int64  
 9   income_w_y     12312 non-null  int64  
 10  wage_w_m       12312 non-null  int64  
 11  wage_w_y       12312 non-null  int64  
 12  perk_w_m       12311 non-null  float64
 13  perk_w_y       12311 non-null  float64
 14  netincome_w_m  12312 non-null  int64  
 15  netincome_w_y  12312 non-null  int64  
 16  Fasl           12312 non-null  int64  
 17  year           12312 non-null  int64  
dtypes: Int

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
همبستگی ستون‌های income_w , wage_w, netincome_w مورد برسی قرار گرفت و چون همستگی ستون income در سال‌های دیگه کم بود فقط ستون wage_w حذف میشود.

In [204]:
DF_P4_S1[['income_w_m', 'wage_w_m', 'netincome_w_m']].corr()

,income_w_m,wage_w_m,netincome_w_m
income_w_m,1.000000,0.857634,0.880028
wage_w_m,0.857634,1.000000,0.975208
netincome_w_m,0.880028,0.975208,1.000000


In [205]:
DF_P4_S1[['income_w_y', 'wage_w_y', 'netincome_w_y']].corr()

,income_w_y,wage_w_y,netincome_w_y
income_w_y,1.000000,0.836300,0.852909
wage_w_y,0.836300,1.000000,0.982256
netincome_w_y,0.852909,0.982256,1.000000


In [206]:
DF_P4_S1.drop(['wage_w_m'], axis=1 , inplace= True)
DF_P4_S1.drop(['wage_w_y'], axis=1 , inplace= True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
با توجه به اطلاعات زیر ستون‌های employed_w, status_w,  perk_w_m دارای imblance هستند بنابراین حذف می‌کنیم.

In [207]:
for col in DF_P4_S1.columns:
 print(f'{col :>15} {(DF_P4_S1[col].value_counts()/len(DF_P4_S1)).max():f}')

        Address 0.000487
         member 0.663255
     employed_w 0.941277
         ISCO_w 0.174301
         ISIC_w 0.170484
       status_w 0.768681
        hours_w 0.533382
         days_w 0.530945
     income_w_m 0.068957
     income_w_y 0.041910
       perk_w_m 0.930474
       perk_w_y 0.580247
  netincome_w_m 0.069120
  netincome_w_y 0.041992
           Fasl 0.252762
           year 1.000000


In [208]:
DF_P4_S1.drop(['employed_w', 'status_w', 'perk_w_m'], inplace=True, axis=1)

In [209]:
DF_P4_S1.isna().sum()

Address            0
member             0
ISCO_w             1
ISIC_w             0
hours_w          697
days_w           697
income_w_m         0
income_w_y         0
perk_w_y           1
netincome_w_m      0
netincome_w_y      0
Fasl               0
year               0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های day_w , hours_w با گروه بندی روی ISCO_w, ISIC_w پر می‌کنیم و ما بقی را با میانه کل داده‌ها پر می‌کنیم.

In [210]:
temp = DF_P4_S1.groupby(['ISCO_w', 'ISIC_w' ])[['hours_w','days_w']].transform('median').astype(pd.Int16Dtype())
DF_P4_S1.loc[DF_P4_S1.hours_w.isna(), 'hours_w'] = temp.loc[DF_P4_S1.hours_w.isna(), 'hours_w']
DF_P4_S1.loc[DF_P4_S1.days_w.isna(), 'days_w'] = temp.loc[DF_P4_S1.days_w.isna(), 'days_w']

In [211]:
DF_P4_S1.isna().sum()

Address           0
member            0
ISCO_w            1
ISIC_w            0
hours_w          70
days_w           70
income_w_m        0
income_w_y        0
perk_w_y          1
netincome_w_m     0
netincome_w_y     0
Fasl              0
year              0
dtype: int64

In [212]:
DF_P4_S1.hours_w.fillna(DF_P4_S1.hours_w.median(),inplace= True)
DF_P4_S1.days_w.fillna(DF_P4_S1.days_w.median(), inplace= True)

In [213]:
DF_P4_S1.dropna(inplace=True)

In [214]:
DF_P4_S1.isna().sum()

Address          0
member           0
ISCO_w           0
ISIC_w           0
hours_w          0
days_w           0
income_w_m       0
income_w_y       0
perk_w_y         0
netincome_w_m    0
netincome_w_y    0
Fasl             0
year             0
dtype: int64

In [215]:
path_file = 'U1401_P4_S01'
DF_P4_S1.to_csv(path_file+'.csv', index=False)

### P4_S02

In [182]:
DF_P4_S2 = pd.read_csv('U1401P4S02.csv')
DF_P4_S2.head()

,Address,member,employed_s,ISCO_s,ISIC_s,status_s,agriculture,hours_s,days_s,cost_employment,cost_raw,cost_machinery,cost_others,cost_tax,sale,income_s_y,Fasl,year,DYCOL00
0,10001000141,1,1,5224,47710,5,2,10,6,0,7050000000,0,720000000,30000000,8460000000,660000000,4,1401,NaN
1,10001001038,1,1,3334,68200,5,2,8,7,0,0,0,10000000,0,2410000000,2400000000,4,1401,NaN
2,10001001428,1,1,6111,1110,5,1,9,5,0,1420000000,0,20000000,0,1840000000,400000000,4,1401,NaN
3,10001002329,1,1,7233,33190,5,2,9,6,0,0,40000000,384000000,0,1144000000,720000000,4,1401,NaN
4,10001002732,1,1,8322,49230,5,2,10,4,0,1900000000,50000000,175000000,0,5500000000,3375000000,4,1401,NaN


In [183]:
DF_P4_S2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6174 entries, 0 to 6173
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Address          6174 non-null   int64  
 1   member           6174 non-null   int64  
 2   employed_s       6174 non-null   int64  
 3   ISCO_s           6174 non-null   int64  
 4   ISIC_s           6174 non-null   int64  
 5   status_s         6174 non-null   int64  
 6   agriculture      6174 non-null   int64  
 7   hours_s          6174 non-null   object 
 8   days_s           6174 non-null   object 
 9   cost_employment  6174 non-null   int64  
 10  cost_raw         6174 non-null   int64  
 11  cost_machinery   6174 non-null   int64  
 12  cost_others      6174 non-null   int64  
 13  cost_tax         6174 non-null   int64  
 14  sale             6174 non-null   int64  
 15  income_s_y       6174 non-null   object 
 16  Fasl             6174 non-null   int64  
 17  year          

In [184]:
DF_P4_S2.drop(['DYCOL00'], axis=1 , inplace= True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون های days_w, hours_s , status_s, income_s_y به int تبدیل می‌کنیم

In [185]:
DF_P4_S2.hours_s.unique()

array(['10', '8 ', '9 ', '5 ', '7 ', '4 ', '6 ', '12', '3 ', '11', '  ',
       '2 ', '1 ', '15', '13', '14', '18', '16', '08', '09'], dtype=object)

In [186]:
DF_P4_S2.loc[DF_P4_S2.hours_s.isin(['  ']), 'hours_s'] = np.nan
DF_P4_S2.hours_s = DF_P4_S2.hours_s.astype(pd.Int64Dtype())

In [187]:
DF_P4_S2.days_s.unique()

array(['6', '7', '5', '4', '2', '3', ' ', '1'], dtype=object)

In [188]:
DF_P4_S2.loc[DF_P4_S2.days_s.isin([' ']), 'days_s'] = np.nan
DF_P4_S2.days_s = DF_P4_S2.days_s.astype(pd.Int64Dtype())

In [189]:
DF_P4_S2.income_s_y = DF_P4_S2.income_s_y.str.strip()

In [190]:
DF_P4_S2.loc[DF_P4_S2.income_s_y.isin(['']), 'income_s_y'] = np.nan
DF_P4_S2.income_s_y = DF_P4_S2.income_s_y.astype(pd.Int64Dtype())

In [191]:
DF_P4_S2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6174 entries, 0 to 6173
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Address          6174 non-null   int64
 1   member           6174 non-null   int64
 2   employed_s       6174 non-null   int64
 3   ISCO_s           6174 non-null   int64
 4   ISIC_s           6174 non-null   int64
 5   status_s         6174 non-null   int64
 6   agriculture      6174 non-null   int64
 7   hours_s          6064 non-null   Int64
 8   days_s           6059 non-null   Int64
 9   cost_employment  6174 non-null   int64
 10  cost_raw         6174 non-null   int64
 11  cost_machinery   6174 non-null   int64
 12  cost_others      6174 non-null   int64
 13  cost_tax         6174 non-null   int64
 14  sale             6174 non-null   int64
 15  income_s_y       6113 non-null   Int64
 16  Fasl             6174 non-null   int64
 17  year             6174 non-null   int64
dtypes: Int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های cost جمع می‌شوند و در ستون جدیدی به نام Total_cost ذخیره می‌شوند.

In [192]:
del_columns = ['cost_employment', 'cost_raw', 'cost_machinery', 'cost_others', 'cost_tax']
DF_P4_S2.insert(9,'Total_cost', DF_P4_S2.loc[:,'cost_employment':'cost_tax' ].sum(axis=1))
DF_P4_S2.drop(del_columns, axis=1, inplace=True)

In [193]:
DF_P4_S2.isna().sum()

Address          0
member           0
employed_s       0
ISCO_s           0
ISIC_s           0
status_s         0
agriculture      0
hours_s        110
days_s         115
Total_cost       0
sale             0
income_s_y      61
Fasl             0
year             0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های day_s , hours_s با گروه بندی روی ISCO_s, ISIC_s پر می‌کنیم و ما بقی را با میانه کل داده‌ها پر می‌کنیم.

In [194]:
temp = DF_P4_S2.groupby(['ISCO_s', 'ISIC_s' ])[['hours_s','days_s']].transform('median')
temp = np.floor(temp).astype(pd.Int64Dtype())
temp.loc[DF_P4_S2.hours_s.isna(), 'hours_s']
DF_P4_S2.loc[DF_P4_S2.hours_s.isna(), 'hours_s']# = temp.loc[DF_P4_S2.hours_s.isna(), 'hours_s']
DF_P4_S2.loc[DF_P4_S2.days_s.isna(), 'days_s'] = temp.loc[DF_P4_S2.days_s.isna(), 'days_s']

DF_P4_S2.hours_s.fillna(DF_P4_S2.hours_s.median(),inplace= True)
DF_P4_S2.days_s.fillna(DF_P4_S2.days_s.median(), inplace= True)

In [195]:
DF_P4_S2.isna().sum()

Address         0
member          0
employed_s      0
ISCO_s          0
ISIC_s          0
status_s        0
agriculture     0
hours_s         0
days_s          0
Total_cost      0
sale            0
income_s_y     61
Fasl            0
year            0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون income_s_y
همان طور که در کد زیر نشان داده شده است تقریبن  در همه موارد nan در ستون status_s مقدار ۶ )کارکن فامیلی) دارد.

In [141]:
DF_P4_S2.loc[DF_P4_S2.income_s_y.isna()].status_s.value_counts()

status_s
6    60
5     1
Name: count, dtype: int64

In [142]:
DF_P4_S2.fillna(0, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون employed_s به دلیل imblance حذف می‌کنیم.

In [143]:
for col in DF_P4_S2.columns:
 print(f'{col :>15} {(DF_P4_S2[col].value_counts()/len(DF_P4_S2)).max():f}')

        Address 0.000972
         member 0.782961
     employed_s 0.977162
         ISCO_s 0.201166
         ISIC_s 0.107548
       status_s 0.864917
    agriculture 0.781341
        hours_s 0.321833
         days_s 0.502268
     Total_cost 0.159054
           sale 0.058633
     income_s_y 0.051506
           Fasl 0.258827
           year 1.000000


In [144]:
DF_P4_S2.drop(['employed_s'],axis=1, inplace=True)

In [145]:
DF_P4_S2.isna().sum()

Address        0
member         0
ISCO_s         0
ISIC_s         0
status_s       0
agriculture    0
hours_s        0
days_s         0
Total_cost     0
sale           0
income_s_y     0
Fasl           0
year           0
dtype: int64

In [159]:
path_file = 'U1401_P4_S02'
DF_P4_S2.to_csv(path_file+'.csv', index=False)

### P4_S03

In [147]:
DF_P4_S3 = pd.read_csv('U1401P4S03.csv')
DF_P4_S3.head()

,Address,member,income_pension,income_rent,income_interest,income_aid,income_resale,income_transfer,Fasl,year,DYCOL00
0,10001000226,1,0,0,0,10000000,0,0,1,1401,NaN
1,10001000230,1,432000000,0,4500000,10000000,0,0,1,1401,NaN
2,10001000232,1,960000000,0,5000000,10000000,0,0,1,1401,NaN
3,10001000232,2,0,0,5000000,0,0,0,1,1401,NaN
4,10001000232,3,0,0,5000000,0,0,0,1,1401,NaN


In [148]:
DF_P4_S3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26783 entries, 0 to 26782
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Address          26783 non-null  int64  
 1   member           26783 non-null  int64  
 2   income_pension   26783 non-null  int64  
 3   income_rent      26783 non-null  int64  
 4   income_interest  26783 non-null  int64  
 5   income_aid       26783 non-null  int64  
 6   income_resale    26783 non-null  int64  
 7   income_transfer  26783 non-null  int64  
 8   Fasl             26783 non-null  int64  
 9   year             26783 non-null  int64  
 10  DYCOL00          0 non-null      float64
dtypes: float64(1), int64(10)
memory usage: 2.2 MB


In [149]:
DF_P4_S3.drop(['DYCOL00'], axis = 1, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های income جمع می‌شوند و در ستون جدیدی به نام Total_income ذخیره می‌شوند.

In [150]:
DF_P4_S3.insert(2,'Total_income', DF_P4_S3.loc[:,'income_pension':'income_transfer' ].sum(axis=1))
del_columns = ['income_pension','income_rent', 'income_interest','income_aid','income_resale','income_transfer']
DF_P4_S3.drop(del_columns,axis=1, inplace=True)

In [151]:
DF_P4_S3.isna().sum()

Address         0
member          0
Total_income    0
Fasl            0
year            0
dtype: int64

In [160]:
path_file = 'U1401_P4_S03'
DF_P4_S3.to_csv(path_file+'.csv', index=False)

### P4_S04

In [153]:
DF_P4_S4 = pd.read_csv('U1401P4S04.csv')
DF_P4_S4.head()

,Address,member,subsidy_number,subsidy_month,subsidy,Fasl,year,DYCOL00
0,10001000126,1,2,4,3640000,4,1401,NaN
1,10001000130,1,1,4,420000,4,1401,NaN
2,10001000132,1,1,4,420000,4,1401,NaN
3,10001000132,2,1,4,1820000,4,1401,NaN
4,10001000135,1,1,4,420000,4,1401,NaN


In [154]:
DF_P4_S4.drop(['DYCOL00'], axis = 1, inplace=True)

In [155]:
DF_P4_S4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30772 entries, 0 to 30771
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Address         30772 non-null  int64
 1   member          30772 non-null  int64
 2   subsidy_number  30772 non-null  int64
 3   subsidy_month   30772 non-null  int64
 4   subsidy         30772 non-null  int64
 5   Fasl            30772 non-null  int64
 6   year            30772 non-null  int64
dtypes: int64(7)
memory usage: 1.6 MB


In [156]:
path_file = 'U1401_P4_S04'
DF_P4_S4.to_csv(path_file+'.csv', index=False)